In [1]:
import math
import numpy as np
import pandas as pd
import datetime
import requests
import re
game_index = pd.read_csv("C:/Users/Roma/Desktop/projects/datasci/allanno.csv") #columns[word, count]

game_index.head()

,ID,player1ID,player2ID,player1Name,player2Name,tourneyID,round,lexicon,uploaded,url
0,1,500,10122,John O'Laughlin,Seth Lipkin,5633,2,TWL06,2019-02-03 20:24:56,https://www.cross-tables.com/annotated/selfgcg...
1,8,10122,66,Seth Lipkin,Joel Horn,5633,5,TWL06,2019-02-03 20:24:56,https://www.cross-tables.com/annotated/selfgcg...
2,9,590,10122,Debbie Stegman,Seth Lipkin,5633,3,TWL06,2019-02-03 20:24:56,https://www.cross-tables.com/annotated/selfgcg...
3,10,1954,1787,Kate Fukawa-Connelly,Dielle Saldanha,5655,8,TWL06,2019-02-03 20:24:56,https://www.cross-tables.com/annotated/selfgcg...
4,11,10122,1598,Seth Lipkin,Ben Dweck,5633,6,TWL06,2019-08-23 15:28:02,https://www.cross-tables.com/annotated/selfgcg...


In [2]:
#Exlpanation on gcg: https://www.poslfit.com/scrabble/gcg/
game_index = game_index[game_index[' uploaded']!="0000-00-00 00:00:00"]

In [4]:
#https://www.geeksforgeeks.org/python-convert-string-to-datetime-and-vice-versa/#
    
format = '%Y-%m-%d %H:%M:%S'
game_index['date_played'] = game_index[' uploaded'].map(lambda a: datetime.datetime.strptime(a, format))
game_index.head()

,ID,player1ID,player2ID,player1Name,player2Name,tourneyID,round,lexicon,uploaded,url,date_played
0,1,500,10122,John O'Laughlin,Seth Lipkin,5633,2,TWL06,2019-02-03 20:24:56,https://www.cross-tables.com/annotated/selfgcg...,2019-02-03 20:24:56
1,8,10122,66,Seth Lipkin,Joel Horn,5633,5,TWL06,2019-02-03 20:24:56,https://www.cross-tables.com/annotated/selfgcg...,2019-02-03 20:24:56
2,9,590,10122,Debbie Stegman,Seth Lipkin,5633,3,TWL06,2019-02-03 20:24:56,https://www.cross-tables.com/annotated/selfgcg...,2019-02-03 20:24:56
3,10,1954,1787,Kate Fukawa-Connelly,Dielle Saldanha,5655,8,TWL06,2019-02-03 20:24:56,https://www.cross-tables.com/annotated/selfgcg...,2019-02-03 20:24:56
4,11,10122,1598,Seth Lipkin,Ben Dweck,5633,6,TWL06,2019-08-23 15:28:02,https://www.cross-tables.com/annotated/selfgcg...,2019-08-23 15:28:02


In [6]:
#turn object
#should include turn number?, player, rack, play-position, word-played, tiles exchanged, tiles drawn, challenge, turn points, total points

#formats to consider:
#regular play.................>David: ANTHER? n8 ANoTHER +73 416
#passed turn..................>Randy: URACK - +0 380
#Tile exchange................>Marlon: SEQSPO? -QO +0 268 
#withdraw phoney..............>Ron: MOULAGD -- -76 354 [76 is the negative of phoney score]
#acceptable challenge bonus...>Joel: DROWNUG (challenge) +5 289 
#end rack points..............>Dave: (G) +4 539 
#last rack (international)....>Pakorn: FWLI (FWLI) -10 426 
#time penalty.................>Pakorn: ISBALI (time) -10 409 


In [7]:
#person object
#PlayerID, name, list of opponent-date-tuples, list of tourneyID's
class Person(object):
    playerID = ""
    name = ""
    opps = []
    tourns = []
    
    def __init__(self, playerID, name, opps, tourns):
        self.playerID = playerID
        self.name = name
        self.opps = opps
        self.tourns = tourns
        
def make_person():
    person = Person(playerID, name, opps, tourns)
    return person

In [8]:
#Game object
class Game(object):
    P1_ID = "" #these can be strings for now, or we can make people objects
    P2_ID = ""
    gameID = ""
    round_num = 0
    turns = [] #this should be a list of turn objects
    winner = ""
    game_date = None

    # The class "constructor" - It's actually an initializer 
    def __init__(self, P1_ID, P2_ID, game_ID, round_num, turns, winner, game_date):
        self.P1_ID = P1_ID
        self.P2_ID = P2_ID
        self.game_ID = game_ID
        self.round_um = round_num
        self.turns = turns
        self.winner = winner
        self.game_date = game_date

def make_game(P1_ID, P2_ID, game_ID, round_num, turns, winner, game_date):
    game = Game(P1_ID, P2_ID, game_ID, round_num, turns, winner, game_date)
    return game

In [35]:
#in our example game would be like testr (r.text from requests object)

def get_turns(game): #takes a gcg string "game" then splits and parses
    all_turns = game.split("\r\n")#[0].split("#")
    embodied_turns = []
    for trn in all_turns:
        if "#" not in trn and trn != "": #this is bad practice, go back and optimize it later
            #print(trn)
            pieces = trn.split(" ") #list format: [Name, Rack, pos(has #, not always present), play (neg means xchnge), score, total_score]
            player_name = pieces[0][1:-1].replace("_", " ") #the name of the player
            rack = pieces[1] #some games only have one player POV

            #pos_res = re.search(r'[0-9]{1,2}[A-Z]|[A-Z][0-9]{1,2}', trn)#check if none
            #play_pos = lambda pos_res: pos_res.group() if pos_res != None else None #this is in error, returns a function
            play_pos = "[not implemented]"
            
            tx_res = re.search(r'-[?, A-Z]+\b', trn) #example of xchng >Seth: GNNOOOV -GOOONV +0 0
            tiles_xchngd = lambda tx_res: tx_res.group() if tx_res != None else None

            word_played = pieces[-3]#also includes exchanged tiles
            challenged = "[not implemented]" #if next turn has a negative score then it's withdrawn, implement this later
            points_earned = pieces[-2]
            total_points = pieces[-1]#.replace("\r\n", "")
            turn_as_dict = {'player_name':player_name,'rack':rack, 'play_pos':play_pos, 'word_played':word_played, 'challenged':challenged, 
                           'points_earned':points_earned, 'total_points':total_points}
            embodied_turns.append(turn_as_dict)
    return embodied_turns

In [36]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0',
    'Accept': '*/*', 
}

url = game_index[' url'][0] #this gets one game, we'll need to loop through all of them...
r = requests.get(url, headers=headers) #well, apparently this works...
print(r.text)

#player1 John_O'Laughlin John O'Laughlin
#player2 Seth Seth
>John_O'Laughlin: IQ 8G QI +22 22
>Seth: GNNOOOV -GOOONV +0 0
#note VOGON* is still not good...
>John_O'Laughlin: IOP 9E POI +18 40
>Seth: EENSSVX 10D VEX +58 58
>John_O'Laughlin: AJOS 11E SOJA +42 82
>Seth: ENNSSTU I9 UNS +16 74
>John_O'Laughlin: AERW H11 .WARE +36 118
>Seth: EEINNST J4 INTENSE +67 141
>John_O'Laughlin: AY G11 .AY +31 149
>Seth: AEHRRTT 12A HEART +30 171
>John_O'Laughlin: GRUU K10 GURU +14 163
>Seth: AERTTTZ A12 .AZE +48 219
>John_O'Laughlin: DEGIMO 5E DEMOI.G +44 207
>Seth: BIRTTTU H1 TURB.T +33 252
>John_O'Laughlin: DFI 6D DIF +31 238
>Seth: CIILPTW 15D TWIC. +11 263
>John_O'Laughlin: ADIILNO 1D DILA.ION +61 299
>Seth: ACILPSY 2J PALSY +44 307
>John_O'Laughlin: ENO 14A .ONE +28 327
>Seth: ?BCDIKO 13J B.CK +24 331
>John_O'Laughlin: ELLORRT M2 .TROLLER +70 397
>Seth: ??DEFIO 3A mODIFiE. +72 403
>John_O'Laughlin: AEEGHMO N10 MAHOE +45 442
>Seth: AGN O9 NAG +11 414
>Seth:  (EV) +10 424



In [37]:
sample0 = (game_index['ID'].sample(n=10, random_state=1)).reset_index()
sample0

,index,ID
0,34841,39776
1,24966,28155
2,13690,15017
3,28592,32171
4,21638,24418
5,4330,4694
6,37027,42855
7,32878,37082
8,31902,35940
9,19929,22530


In [38]:
#How to access each of the games data
#pcp file something like this: https://www.cross-tables.com/annotated/selfgcg/1/anno122.gcg
#special thanks: https://stackoverflow.com/questions/645312/what-is-the-quickest-way-to-http-get-in-python
#https://www.cross-tables.com/annotated/selfgcg/1/anno122.gcg
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0',
    'Accept': '*/*', 
}

ID_and_turns = {}
count = 0
for game_i in sample0['ID']:
    url = 'https://www.cross-tables.com/annotated/selfgcg/'+str(int(game_i)//100)+'/anno'+str(game_i)+'.gcg'
    rtext = requests.get(url, headers=headers).text #well, apparently this works...
    gID = game_i
    ID_and_turns[gID] = get_turns(rtext)


In [40]:
#ID_and_turns is a dict, with GameID as key and a list of dicts (each dict rep a single turn) as value
(ID_and_turns)

{39776: [{'player_name': 'Morris Greenberg',
   'rack': 'EGIOUUW',
   'play_pos': '[not implemented]',
   'word_played': '-GIOUUW',
   'challenged': '[not implemented]',
   'points_earned': '+0',
   'total_points': '0'},
  {'player_name': 'Steve Ozorio',
   'rack': 'ADHNO',
   'play_pos': '[not implemented]',
   'word_played': 'HONDA',
   'challenged': '[not implemented]',
   'points_earned': '+26',
   'total_points': '26'},
  {'player_name': 'Morris Greenberg',
   'rack': 'AEEIORT',
   'play_pos': '[not implemented]',
   'word_played': 'OE',
   'challenged': '[not implemented]',
   'points_earned': '+9',
   'total_points': '9'},
  {'player_name': 'Steve Ozorio',
   'rack': 'CGO',
   'play_pos': '[not implemented]',
   'word_played': 'COG',
   'challenged': '[not implemented]',
   'points_earned': '+18',
   'total_points': '44'},
  {'player_name': 'Morris Greenberg',
   'rack': 'AEEIIRT',
   'play_pos': '[not implemented]',
   'word_played': 'AI',
   'challenged': '[not implemented]',


In [ ]:
#Two files, one of people and one of games which includes turn structures

#Person
##playerID [str]
##name [str]
##opps [list] (list of player ID's)
##tourns [list] (list of game ID's)

#Game
##P1_ID [str]
##P2_ID [str]
##gameID [str]
##round_num [int]
##turns [list] (list of turns)
###turn0
###turn1
###turn...
##winner [str]
##game_date [datetime object]

#Turn
##player [str] (player ID)
##rack [list]
##play_pos [str]
##word_played [str]
##tiles_xchanged [list]
##challenged [bool]
##points_earned [int]
##total_points [int]


In [ ]:
#implement custer detection